In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
RS = sum(list(map(ord, 'Dale Boca')))

import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

import numpy as np
import os;

### Preparación de datasets

In [2]:
from sklearn.preprocessing import LabelEncoder

def dataset_prep(window=3, neutral=True):
    
    train = pd.read_csv('data/trainset.csv', index_col=0)
    test  = pd.read_csv('data/testset.csv', index_col=0)
    
    cols1 = ['pos', 'neg', 'neu'] if neutral==True else ['pos', 'neg']
    
    for col in cols1:
        train[col+'w'] = train[col].rolling(window=window).sum()
        test[col+'w'] = test[col].rolling(window=window).sum()
    
    train['exc_ret'] = train['exc_ret'].shift(periods=-1)
    test['exc_ret']  = test['exc_ret'].shift(periods=-1)
    
    train.dropna(axis=0, inplace=True)
    test.dropna(axis=0, inplace=True)
    
    cols2 = [x+'w' for x in cols1]
    
    X = train[cols2]
    y = train.exc_ret
    Xt = test[cols2]
    yt = test.exc_ret
    
    le = LabelEncoder()
    y  = le.fit_transform(y)
    yt = le.fit_transform(yt)
    
    return X, Xt, y, yt    

### Entrenamiento de Clasificadores

In [3]:
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from scikitplot.metrics import plot_roc

from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier 

In [4]:
def train_classifier(X, Xt, y, yt):
    
    model_name = ['Random Forest', 'XGBoost', 'LightGBM', 'Logistic Regression', 'Catboost']
   
    model_init = [RandomForestClassifier(),
                  XGBClassifier(),
                  lgb.LGBMClassifier(),
                  LogisticRegression(),
                  CatBoostClassifier()]
    params1 = {  
        "n_estimators": st.randint(10,100),    # Number of boosted trees to fit.
        "max_depth"   : st.randint(2, 25),     # Maximum tree depth for base learners.
    }
    
    params2 = {  
        "n_estimators": st.randint(10,100),    # Number of boosted trees to fit.
        "max_depth": st.randint(2, 25),         # Maximum tree depth for base learners.
        "learning_rate": st.uniform(0.01, 0.5), # Boosting learning rate (xgb’s “eta”)
        "colsample_bytree": st.beta(10, 1),     # Subsample ratio of columns when constructing each tree.
        "subsample": st.beta(10, 1),            # Subsample ratio of the training instance.
        "gamma": st.uniform(0, 10),             # Minimum loss reduction required to make a further partition on a leaf node of the tree.
        'reg_alpha': st.uniform(0.05,10),       # L1 regularization term on weights
        "min_child_weight": st.uniform(1,20),   # Minimum sum of instance weight(hessian) needed in a child.
    }
    
    params3 = {  
        "penalty"     : ['l1', 'l2'],
        "C"           : st.uniform(0.1, 10.),
    }
    
    params4 = {}
    
    model_params = [params1, params2, params2, params3, params4]
    
    train_scores    = []
    test_scores     = []
    best_estimators = []
    best_parameters = []
    
    for name, mod, params in zip(model_name, model_init, model_params):
        model = mod
        kf    = KFold(n_splits=5, shuffle=True, random_state=RS)
        rgrid = RandomizedSearchCV(estimator=model, param_distributions=params, cv=kf,
                                   scoring='roc_auc', n_iter=50, verbose=1, n_jobs=-1)
        rgrid.fit(X, y)
        best_estimators.append(rgrid.best_estimator_)
        best_parameters.append(rgrid.best_params_)
        train_scores.append(rgrid.best_score_)
        ppred = rgrid.predict_proba(Xt)
        score = roc_auc_score(y_score=ppred[:, 1], y_true=yt)
        test_scores.append(score)
    
    results = pd.DataFrame(
        {'Model'      : model_name,
         'Train Score': train_scores,
         'Test Score' : test_scores,
         'Params'     : best_parameters,
         'Estimator'  : best_estimators
        })
    
    return results.sort_values(by='Test Score', ascending=False)

In [5]:
import itertools
windows = np.arange(1, 6)
neutral = [1, 0]

resultado = pd.DataFrame()

for win, neu in itertools.product(windows, neutral):
    
    X, X_test, y, y_test = dataset_prep(window=win, neutral=neu)
    
    clfs = train_classifier(X, X_test, y, y_test)
  
    res = pd.DataFrame(
    {'Model'      : clfs['Model'],
     'Window'     : list(str(win)*clfs.shape[0]),
     'Neutral'    : list(str(neu)*clfs.shape[0]),
     'Train Score': clfs['Train Score'],
     'Test Score' : clfs['Test Score'],
     'Params'     : clfs['Params'],
     'Estimator'  : clfs['Estimator']
    })
    
    resultado = pd.concat([resultado, res], axis=0)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    9.7s finished
/anaconda3/envs/DataScience36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.4s finished
/anaconda3/envs/DataScience36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 50 candidates, totalling 250 fits


...

337:	learn: 0.5731529	total: 2.81s	remaining: 5.49s
338:	learn: 0.5729734	total: 2.81s	remaining: 5.48s
339:	learn: 0.5727928	total: 2.82s	remaining: 5.47s
340:	learn: 0.5726325	total: 2.83s	remaining: 5.46s
341:	learn: 0.5720394	total: 2.83s	remaining: 5.45s
342:	learn: 0.5719879	total: 2.84s	remaining: 5.44s
343:	learn: 0.5718406	total: 2.85s	remaining: 5.43s
344:	learn: 0.5716814	total: 2.85s	remaining: 5.42s
345:	learn: 0.5715437	total: 2.86s	remaining: 5.41s
346:	learn: 0.5714344	total: 2.87s	remaining: 5.39s
347:	learn: 0.5713219	total: 2.88s	remaining: 5.39s
348:	learn: 0.5710563	total: 2.88s	remaining: 5.38s
349:	learn: 0.5707687	total: 2.89s	remaining: 5.37s
350:	learn: 0.5707687	total: 2.89s	remaining: 5.35s
351:	learn: 0.5707591	total: 2.9s	remaining: 5.34s
352:	learn: 0.5706003	total: 2.91s	remaining: 5.33s
353:	learn: 0.5704526	total: 2.92s	remaining: 5.32s
354:	learn: 0.5703301	total: 2.92s	remaining: 5.31s
355:	learn: 0.5701646	total: 2.93s	remaining: 5.3s
356:	learn: 0.

503:	learn: 0.5456493	total: 4s	remaining: 3.93s
504:	learn: 0.5454366	total: 4s	remaining: 3.92s
505:	learn: 0.5452424	total: 4.01s	remaining: 3.91s
506:	learn: 0.5452086	total: 4.01s	remaining: 3.9s
507:	learn: 0.5451064	total: 4.02s	remaining: 3.9s
508:	learn: 0.5450894	total: 4.03s	remaining: 3.89s
509:	learn: 0.5448608	total: 4.04s	remaining: 3.88s
510:	learn: 0.5448174	total: 4.05s	remaining: 3.87s
511:	learn: 0.5445496	total: 4.05s	remaining: 3.86s
512:	learn: 0.5442983	total: 4.06s	remaining: 3.85s
513:	learn: 0.5442312	total: 4.07s	remaining: 3.84s
514:	learn: 0.5441724	total: 4.07s	remaining: 3.84s
515:	learn: 0.5439791	total: 4.08s	remaining: 3.83s
516:	learn: 0.5438881	total: 4.09s	remaining: 3.82s
517:	learn: 0.5438857	total: 4.09s	remaining: 3.81s
518:	learn: 0.5437255	total: 4.1s	remaining: 3.8s
519:	learn: 0.5436040	total: 4.11s	remaining: 3.79s
520:	learn: 0.5433300	total: 4.12s	remaining: 3.78s
521:	learn: 0.5433196	total: 4.12s	remaining: 3.77s
522:	learn: 0.5432642	

662:	learn: 0.5238126	total: 5.17s	remaining: 2.63s
663:	learn: 0.5237091	total: 5.18s	remaining: 2.62s
664:	learn: 0.5236971	total: 5.19s	remaining: 2.62s
665:	learn: 0.5236322	total: 5.2s	remaining: 2.61s
666:	learn: 0.5235676	total: 5.21s	remaining: 2.6s
667:	learn: 0.5234437	total: 5.22s	remaining: 2.59s
668:	learn: 0.5233942	total: 5.23s	remaining: 2.59s
669:	learn: 0.5232744	total: 5.23s	remaining: 2.58s
670:	learn: 0.5231549	total: 5.24s	remaining: 2.57s
671:	learn: 0.5230802	total: 5.25s	remaining: 2.56s
672:	learn: 0.5228764	total: 5.25s	remaining: 2.55s
673:	learn: 0.5227934	total: 5.26s	remaining: 2.54s
674:	learn: 0.5227537	total: 5.27s	remaining: 2.54s
675:	learn: 0.5225105	total: 5.27s	remaining: 2.53s
676:	learn: 0.5221307	total: 5.28s	remaining: 2.52s
677:	learn: 0.5220857	total: 5.29s	remaining: 2.51s
678:	learn: 0.5220856	total: 5.29s	remaining: 2.5s
679:	learn: 0.5220166	total: 5.3s	remaining: 2.5s
680:	learn: 0.5219430	total: 5.31s	remaining: 2.49s
681:	learn: 0.521

826:	learn: 0.4992610	total: 6.34s	remaining: 1.33s
827:	learn: 0.4986669	total: 6.35s	remaining: 1.32s
828:	learn: 0.4984669	total: 6.35s	remaining: 1.31s
829:	learn: 0.4984466	total: 6.36s	remaining: 1.3s
830:	learn: 0.4982897	total: 6.37s	remaining: 1.29s
831:	learn: 0.4982017	total: 6.37s	remaining: 1.29s
832:	learn: 0.4979855	total: 6.38s	remaining: 1.28s
833:	learn: 0.4979609	total: 6.39s	remaining: 1.27s
834:	learn: 0.4978993	total: 6.39s	remaining: 1.26s
835:	learn: 0.4977470	total: 6.4s	remaining: 1.25s
836:	learn: 0.4975043	total: 6.41s	remaining: 1.25s
837:	learn: 0.4973732	total: 6.42s	remaining: 1.24s
838:	learn: 0.4972496	total: 6.42s	remaining: 1.23s
839:	learn: 0.4971809	total: 6.43s	remaining: 1.22s
840:	learn: 0.4970183	total: 6.43s	remaining: 1.22s
841:	learn: 0.4968154	total: 6.44s	remaining: 1.21s
842:	learn: 0.4966436	total: 6.45s	remaining: 1.2s
843:	learn: 0.4965940	total: 6.46s	remaining: 1.19s
844:	learn: 0.4964603	total: 6.46s	remaining: 1.19s
845:	learn: 0.4

992:	learn: 0.4798333	total: 7.51s	remaining: 53ms
993:	learn: 0.4796017	total: 7.52s	remaining: 45.4ms
994:	learn: 0.4794560	total: 7.53s	remaining: 37.8ms
995:	learn: 0.4793402	total: 7.53s	remaining: 30.3ms
996:	learn: 0.4791866	total: 7.54s	remaining: 22.7ms
997:	learn: 0.4791278	total: 7.55s	remaining: 15.1ms
998:	learn: 0.4790906	total: 7.56s	remaining: 7.57ms
999:	learn: 0.4786198	total: 7.57s	remaining: 0us


In [7]:
resultado.sort_values(by='Train Score', ascending=False).iloc[:, :-1]

,Model,Window,Neutral,Train Score,Test Score,Params
2,LightGBM,2,1,0.597340,0.440947,"{'colsample_bytree': 0.8719584647440835, 'gamma': 8.691992841401364, 'learning_rate': 0.08424182221663505, 'max_depth': 23, 'min_child_weight': 4.370425544889302, 'n_estimators': 97, 'reg_alpha': 8.711227205114612, 'subsample': 0.9772611466168699}"
0,Random Forest,1,0,0.594299,0.367895,"{'max_depth': 2, 'n_estimators': 97}"
2,LightGBM,3,1,0.594131,0.448617,"{'colsample_bytree': 0.9926655910010295, 'gamma': 0.9577082180196517, 'learning_rate': 0.14491142787845956, 'max_depth': 2, 'min_child_weight': 8.239590874512295, 'n_estimators': 14, 'reg_alpha': 5.482667904352924, 'subsample': 0.9703217198886533}"
0,Random Forest,2,1,0.589667,0.450195,"{'max_depth': 23, 'n_estimators': 28}"
0,Random Forest,3,1,0.589199,0.441864,"{'max_depth': 3, 'n_estimators': 66}"
1,XGBoost,3,1,0.587691,0.458827,"{'colsample_bytree': 0.9815501305656954, 'gamma': 0.053407634012030414, 'learning_rate': 0.10452850922594074, 'max_depth': 12, 'min_child_weight': 9.20888672510881, 'n_estimators': 35, 'reg_alpha': 9.612664396260259, 'subsample': 0.9401579505211932}"
2,LightGBM,4,1,0.584134,0.443603,"{'colsample_bytree': 0.9372578351560642, 'gamma': 7.079538184244899, 'learning_rate': 0.0800997770915091, 'max_depth': 22, 'min_child_weight': 14.742109879080788, 'n_estimators': 43, 'reg_alpha': 4.519453210321728, 'subsample': 0.975363441915842}"
4,Catboost,2,1,0.578966,0.452304,{}
0,Random Forest,2,0,0.578544,0.411259,"{'max_depth': 4, 'n_estimators': 91}"
1,XGBoost,4,1,0.578314,0.462458,"{'colsample_bytree': 0.8942646625270874, 'gamma': 0.2322263420598747, 'learning_rate': 0.33414943746077425, 'max_depth': 8, 'min_child_weight': 8.047092822894289, 'n_estimators': 48, 'reg_alpha': 6.692956528532755, 'subsample': 0.5802009307111863}"
